# Análisis exploratorio de datos de la English Premier League

Importo librerias

In [12]:
from models import *
import pandas as pd
import numpy as np
import warnings
import os

warnings.filterwarnings("ignore")

## Mis hipótesis:
### 1. Es muy difícil competir en la Premier League
### 2. Los equipos necesitan tener inversiones millonarias para competir en la liga

## Recolección de datos

Los datasets utilizados en su mayoria son obtenidos mediante web scraping que explico en la siguiente celda y están ubicados en la carpeta temporada, el otro dataset llamado ```EPL Standings 2010-2021.csv``` fue obtenido del sitio web [Kaggle](https://www.kaggle.com/)

Para obtener los datos scrapeo la web de [Transfermarkt](https://https://www.transfermarkt.es/), reconocida web que recopila todo tipo de datos de jugadores, equipos, ligas, competiciones, agentes y demás. Especificamente ataco al apartado que tiene los datos históricos (Equipos, ingresos, gastos y valores de plantillas en mi caso) de la Premier League y los proceso para que lleguen limpios con las funciones definidas en el fichero ```models.py```

Ejecuto las funciones de scrapeo por cada temporada a analizar y las declaro como variables para posteriormente limpiarlas y convertirlas en archivos.csv para posterior uso (las dejo comentadas para que no se vuelvan a ejecutar por accidente)

In [13]:
# Scrapers().transferscrap_money(2010).to_csv("datasets/temporada/temp_2010_2011_money.csv")
# Scrapers().transferscrap_money(2011).to_csv("datasets/temporada/temp_2011_2012_money.csv")
# Scrapers().transferscrap_money(2012).to_csv("datasets/temporada/temp_2012_2013_money.csv")
# Scrapers().transferscrap_money(2013).to_csv("datasets/temporada/temp_2013_2014_money.csv")
# Scrapers().transferscrap_money(2014).to_csv("datasets/temporada/temp_2014_2015_money.csv")
# Scrapers().transferscrap_money(2015).to_csv("datasets/temporada/temp_2015_2016_money.csv")
# Scrapers().transferscrap_money(2016).to_csv("datasets/temporada/temp_2016_2017_money.csv")
# Scrapers().transferscrap_money(2017).to_csv("datasets/temporada/temp_2017_2018_money.csv")
# Scrapers().transferscrap_money(2018).to_csv("datasets/temporada/temp_2018_2019_money.csv")
# Scrapers().transferscrap_money(2019).to_csv("datasets/temporada/temp_2019_2020_money.csv")
# Scrapers().transferscrap_money(2020).to_csv("datasets/temporada/temp_2020_2021_money.csv")

# Scrapers().transferscrap(2010).to_csv("datasets/temporada/temp_2010_2011.csv")
# Scrapers().transferscrap(2011).to_csv("datasets/temporada/temp_2011_2012.csv")
# Scrapers().transferscrap(2012).to_csv("datasets/temporada/temp_2012_2013.csv")
# Scrapers().transferscrap(2013).to_csv("datasets/temporada/temp_2013_2014.csv")
# Scrapers().transferscrap(2014).to_csv("datasets/temporada/temp_2014_2015.csv")
# Scrapers().transferscrap(2015).to_csv("datasets/temporada/temp_2015_2016.csv")
# Scrapers().transferscrap(2016).to_csv("datasets/temporada/temp_2016_2017.csv")
# Scrapers().transferscrap(2017).to_csv("datasets/temporada/temp_2017_2018.csv")
# Scrapers().transferscrap(2018).to_csv("datasets/temporada/temp_2018_2019.csv")
# Scrapers().transferscrap(2019).to_csv("datasets/temporada/temp_2019_2020.csv")
# Scrapers().transferscrap(2020).to_csv("datasets/temporada/temp_2020_2021.csv")

## Estudio de los datos

Defino la ruta de origen a los datasets

In [14]:
PATH = f"{os.path.abspath('')}\\datasets\\"

Leo la información que voy a necesitar

In [15]:
standings = pd.read_csv(PATH + "EPL Standings 2010-2021.csv").drop(["Qualification or relegation"], axis=1)

In [16]:
value_data = []
money_data = []

for dataset in os.listdir(PATH + "temporada\\"):
    if dataset[-9:] == "money.csv":
        df = pd.read_csv(PATH + "temporada\\" + dataset).drop(["Unnamed: 0"], axis=1)
        money_data.append(df)
    else:
        df = pd.read_csv(PATH + "temporada\\" + dataset).drop(["Unnamed: 0"], axis=1)
        value_data.append(df)

Luego emparejo los dataframes leídos con sus registros correspondientes en el dataframe standings

In [17]:
contador = 0
team_value_season = []

for i in range(0, 11):
    value = pd.DataFrame(value_data[i])
    money = pd.DataFrame(money_data[i])
    standings_2 = pd.DataFrame(standings[contador:(contador+20)])
    contador += 20
    df = standings_2.merge(value, right_on=["Team", "Season"], left_on=["Team", "Season"], how="outer")
    team_value_season.append(df.merge(money, right_on=["Team", "Season"], left_on=["Team", "Season"], how="outer"))

### Comenzamos con la hipótesis 1

Calculamos la dificultad de la liga en estos 3 aspectos: 
1. Salir campeón
2. Clasificarse a competiciones europeas (Champions League y Europa League) 
3. Mantener la categoría

Enseño el rendimineto a largo plazo de cada equipo en base a sus puntos, goles a favor y victorias

In [18]:
total_standings = pd.concat([i for i in team_value_season], ignore_index=True, axis=0)
total_standings["balance"] = total_standings["profits"] - total_standings["losses"] 
Graficas.hist_(titulo="Rendimiento a largo plazo de los equipos (Puntos)", df=total_standings, x="Pts", y="Team", orient="h")
Graficas.hist_(titulo="Rendimiento a largo plazo de los equipos (Goles)", df=total_standings, x="GF", y="Team", orient="h")
Graficas.hist_(titulo="Rendimiento a largo plazo de los equipos (Victorias)", df=total_standings, x="W", y="Team", orient="h")

Separo los campeones y calculo al campeón promedio

In [19]:
campeones = Competicion.promedio_temporadas(df=total_standings, competicion="Campeón de liga", lista=[1])
Graficas.box_(titulo="Equipos campeones de liga", df=campeones, x="Team", y="Pts")

Separo los clasificados a la Champions League y calculo al clasificado promedio

In [20]:
champions = Competicion.promedio_temporadas(df=total_standings, competicion="UEFA Champions League", lista=[1, 4])
Graficas.hist_(titulo="Equipos clasificados a UEFA Champions League", df=champions, x=["Team", "Season"], y="Pts", columnas=["Pos"])
Graficas.box_(titulo="Equipos clasificados a UEFA Champions League", df=champions, x="Season", y="Pts")

Separo los clasificados a la Europa League y calculo al clasificado promedio

In [21]:
europa = Competicion.promedio_temporadas(df=total_standings, competicion="UEFA Europa League", lista=[5])
Graficas.box_(titulo="Equipos clasificados a UEFA Europa League", df=europa, x="Team", y="Pts")

Separo a los últimos 3 equipos por encima de los puestos de descenso de la tabla de cada temporada 

In [22]:
descenso = Competicion.promedio_temporadas(df=total_standings, competicion="Salvados del descenso", lista=[15, 17])
Graficas.hist_(titulo="Equipos salvados del descenso", df=descenso, x=["Team", "Season"], y="Pts")
Graficas.box_(titulo="Equipos salvados del descenso", df=descenso, x="Season", y="Pts")

Comparo la distancia entre el Campeón y el 2do y 3er clasificado para medir la dificultad de ser campeón

In [23]:
segundo_tercero = Competicion.promedio_temporadas(df=total_standings, competicion="2do y 3er clasificado", lista=[2, 3])
dif_campeon = pd.concat([campeones, segundo_tercero], ignore_index=True, axis=0)
Graficas.box_(titulo="Dificultad de salir campeón de liga", df=dif_campeon, x="Season", y="Pts")

Comparo la distancia entre el 3er y 4to clasificado (últimos boletos a la Champions league) contra el 5to (clasificado a Europa League) y el 6to para medir la dificultad de poder entrar en la Liga de campeones

In [24]:
tercero_cuarto = Competicion.promedio_temporadas(df=total_standings, competicion="3er y 4to clasificado", lista=[3, 4])
sextos = Competicion.promedio_temporadas(df=total_standings, competicion="6to clasificado", lista=[6])
dif_champions = pd.concat([tercero_cuarto, europa, sextos], ignore_index=True, axis=0)
Graficas.box_(titulo="Dificultad de clasificarse a competiciones Europeas", df=dif_champions, x="Season", y="Pts")

Comparo la distancia entre los 3 primeros y los 3 salvados del descenso para medir la dificultad de la liga basándome en que mientras más difícil es vencer a los equipos de las posiciones más altas, más difícil es competir

In [25]:
dif_liga = pd.concat([dif_campeon, descenso], ignore_index=True, axis=0)
Graficas.box_(titulo="Dificultad de la liga", df=dif_liga, x="Season", y="Pts")

### Continuamos con la hipótesis 2

Comprobamos la correlación que existe entre el gasto en fichajes y el valor de las plantillas

In [26]:
value_money = total_standings.groupby(["Season"])[["balance", "Value mill €"]].head()
Graficas.heat_(titulo="Correlación entre balance de fichajes y valor de plantilla", df=value_money, x="balance", y="Value mill €")

Deducimos cuales son los equipos que revalorizan más a sus jugadores

In [27]:
revaloriza = total_standings.groupby(["Team"])[["Value mill €", "balance"]].mean().reset_index()
Graficas.scatter_(titulo="Valor promedio de plantillas contra balance de fichajes", df=revaloriza, x="Value mill €", y="balance")

Saco los promedios del valor de las plantillas y el balance de fichajes de todos los equipos por temporada

In [28]:
mean_value_money_per_season = total_standings.groupby(["Season"])[["Value mill €", "balance"]].mean().reset_index()
Graficas.hist_(titulo="Valor promedio de plantillas por temporadas", df=mean_value_money_per_season, x="Season", y="Value mill €")
Graficas.hist_(titulo="Balance de fichajes de equipos por temporadas", df=mean_value_money_per_season, x="Season", y="balance")
Graficas.box_(titulo="Valor promedio de plantillas por temporadas", df=total_standings, x="Season", y="Value mill €")
Graficas.box_(titulo="Balance de fichajes por temporadas", df=total_standings, x="Season", y="balance")

Muestro el valor promedio de las plantillas y el balance de fichajes por equipo

In [29]:
value_per_team = np.round(total_standings.groupby(["Team"])[["Value mill €", "balance"]].mean().reset_index().sort_values
(by="Value mill €", ascending=False))
Graficas.box_(titulo="Valor promedio de plantillas por equipo", df=value_per_team, x="Value mill €", y="Team")
Graficas.box_(titulo="Balance de fichajes por equipo", df=value_per_team, x="balance", y="Team")

Muestro el valor promedio de las plantillas y el balance de fichajes de los equipos que han quedado en esa posición en la tabla

In [30]:
value_money_per_pos = total_standings.groupby(["Pos"])[["Value mill €", "balance"]].mean().reset_index()
Graficas.box_(titulo="Valor de plantilla por posición", df=value_money_per_pos, x="Pos", y="Value mill €")
Graficas.box_(titulo="Balance de fichajes por posición", df=value_money_per_pos, x="Pos", y="balance")

Muestro el valor promedio de las plantillas de cada equipo y su balance de fichajes cada vez que ese equipo quedó en una posición determinada

In [31]:
mean_money_value_team = np.round(total_standings.groupby(["Pos", "Season", "Team"])[["Value mill €", "balance"]].mean(), 3).reset_index()
Graficas.box_(titulo="Valor de plantilla por equipo en una posición determinada", df=mean_money_value_team, x="Pos", y="Value mill €")
Graficas.box_(titulo="Balance de fichajes por equipo en una posición determinada", df=mean_money_value_team, x="Pos", y="balance")

Mostramos los valores de plantilla que los equipos tuvieron por año y en que posición quedaron

In [32]:
team_value = np.round(total_standings.groupby(["Season", "Team", "Pos", "Pts"])[["Value mill €"]].max(), 3).reset_index()
Graficas.box_(titulo="Valor de plantilla por equipo por año", df=team_value, x="Season", y="Value mill €")

Mostramos los balances de fichajes que los equipos tuvieron por año y en que posición quedaron

In [33]:
team_balance = np.round(total_standings.groupby(["Season", "Team", "Pos", "Pts"])[["balance"]].mean(), 3).reset_index()
Graficas.box_(titulo="Balance de fichajes por equipo por año", df=team_balance, x="Season", y="balance")

Mostramos el coste promedio de cada punto obtenido por cada equipo y el coste promedio de cada punto en la liga en las temporadas estudiadas

In [35]:
coste_punto = Competicion.data_coste(elemento="Pts", df=total_standings)
Graficas.hist_(titulo="Coste de puntos por equipo", df=coste_punto, x="mill/Pts", y="Team", orient="h", columnas=["balance", "Pos"])
Graficas.scatter_(titulo="Coste de puntos por equipo", df=coste_punto, x="Pts", y="balance")

Mostramos el coste promedio de goles a favor en la liga en las temporadas estudiadas

In [36]:
coste_goles = Competicion.data_coste(elemento="GF", df=total_standings)
Graficas.hist_(titulo="Coste de goles por equipo", df=coste_goles, x="mill/GF", y="Team", orient="h", columnas=["balance", "Pos"])
Graficas.scatter_(titulo="Coste de goles por equipo", df=coste_goles, x="GF", y="balance")

Mostramos el valor de cada victoria en la liga de las temporadas estudiadas

In [37]:
coste_victoria = Competicion.data_coste(elemento="W", df=total_standings)
Graficas.hist_(titulo="Coste de victorias por equipo", df=coste_victoria, x="mill/W", y="Team", orient="h", columnas=["balance", "Pos"])
Graficas.scatter_(titulo="Coste de victorias por equipo", df=coste_victoria, x="W", y="balance")